In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(category=FutureWarning, action="ignore")

In [3]:
def preprocess(data_path, val_col):
    df = pd.read_csv(data_path)
    df = df.rename(columns={"TIME": "ds", val_col: "y"})
    df['ds'] = pd.to_datetime(df['ds'])
    
    print(df.head())

    return df

price_df = preprocess("data/price.csv", "PRICE")
buy_energy_df = preprocess("data/purch_energy.csv", "ENERGY")
sell_energy_df = preprocess("data/sale_energy.csv", "ENERGY")

                   ds      y
0 2024-01-01 00:00:00  63.33
1 2024-01-01 00:15:00  60.02
2 2024-01-01 00:30:00  56.71
3 2024-01-01 00:45:00  53.40
4 2024-01-01 01:00:00  50.09
                   ds            y
0 2024-01-01 00:00:00   380.176396
1 2024-01-01 00:15:00   291.550179
2 2024-01-01 00:30:00  2982.256610
3 2024-01-01 00:45:00  1541.116815
4 2024-01-01 01:00:00   364.479790
                   ds            y
0 2024-01-01 00:00:00   734.649629
1 2024-01-01 00:15:00  1198.827344
2 2024-01-01 00:30:00   346.984806
3 2024-01-01 00:45:00  1788.538221
4 2024-01-01 01:00:00   989.350092


In [7]:
def train(df):
    folds = NeuralProphet(n_lags=8, n_forecasts=8)\
        .crossvalidation_split_df(df, freq='15min', k=5, fold_pct=0.2, fold_overlap_pct=0.5)

    METRICS = ["MAE", "RMSE"]
    METRICS_VAL = ["MAE_val", "RMSE_val"]

    metrics_train = pd.DataFrame(columns=METRICS)
    metrics_test = pd.DataFrame(columns=METRICS_VAL)

    for df_train, df_test in folds:
        m = NeuralProphet(n_lags=8, n_forecasts=8)
        train = m.fit(df=df_train, freq="15min", validation_df=df_test)
        test = m.test(df=df_test)
        metrics_train = metrics_train.append(train[METRICS].iloc[-1])
        metrics_test = metrics_test.append(test[METRICS_VAL].iloc[-1])

    print(metrics_test.describe().loc[["mean", "std", "min", "max"]])

    return m, metrics_train, metrics_test


def predict(model, df, scaler=None, forecast_hours=24):
    periods = int((forecast_hours * 60) / 15)
    future = model.make_future_dataframe(df, periods=periods)

    forecast = model.predict(future)

    if scaler is not None:
        forecast['yhat1'] = scaler.inverse_transform(forecast[['yhat1']])
        if 'y' in forecast.columns:
            forecast['y'] = scaler.inverse_transform(forecast[['y']])

    return forecast

In [ ]:
train(price_df)